In [1]:
import os
import psycopg2

hostname = os.environ.get('DB_HOSTNAME')
database = os.environ.get('DB_DATABASE')
username = os.environ.get('DB_USERNAME')
pwd = os.environ.get('DB_PWD')
port = int(os.environ.get('DB_PORT'))


conn = psycopg2.connect(host=hostname, dbname=database, user=username, password=pwd, port=port)



In [8]:
import pandas as pd

symbol = 'BTCUSDT'
input_path = f'../outputs/{symbol}_upload.csv'

df = pd.read_csv(input_path)
df.head()

,open_time,open_price,high_price,low_price,close_price,volume,close_time,quote_asset_volume,number_of_trades,symbol,pk,should_be_updated
0,2024-01-17 04:00:00,42849.00,42921.10,42781.49,42854.01,791.40,2024-01-17 04:59:59.999,33911017.91,32838,BTCUSDT,2024-01-17 04:00:00_btcusdt_2024-01-17 04:59:5...,False
1,2024-01-17 05:00:00,42854.01,42925.49,42765.41,42783.39,661.25,2024-01-17 05:59:59.999,28333635.53,30696,BTCUSDT,2024-01-17 05:00:00_btcusdt_2024-01-17 05:59:5...,False
2,2024-01-17 06:00:00,42783.39,42876.34,42766.00,42806.05,802.28,2024-01-17 06:59:59.999,34360257.70,28881,BTCUSDT,2024-01-17 06:00:00_btcusdt_2024-01-17 06:59:5...,False
3,2024-01-17 07:00:00,42806.05,42851.10,42619.60,42655.94,1357.31,2024-01-17 07:59:59.999,58019581.23,46737,BTCUSDT,2024-01-17 07:00:00_btcusdt_2024-01-17 07:59:5...,True
4,2024-01-17 08:00:00,42655.95,42747.65,42564.00,42728.76,1188.19,2024-01-17 08:59:59.999,50705137.24,52602,BTCUSDT,2024-01-17 08:00:00_btcusdt_2024-01-17 08:59:5...,True


In [11]:
new_data_df = df[df['should_be_updated'] == False]
# new_data_df
new_data_df = new_data_df[new_data_df.columns[:-2]]
new_data_df['symbol'] = new_data_df['symbol'].str.lower()

In [5]:
import os
from sqlalchemy import create_engine

hostname = os.environ.get('DB_HOSTNAME')
database = os.environ.get('DB_DATABASE')
username = os.environ.get('DB_USERNAME')
pwd = os.environ.get('DB_PWD')
port = int(os.environ.get('DB_PORT'))


db_params = {
    'host': hostname,
    'user': username,
    'password': pwd,
    'database': database,
    'port': port,
}

db_url = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["database"]}'


engine = create_engine(db_url)


In [12]:
new_data_df.to_sql(name='klines', con=engine, if_exists='append', index=False, method='multi')

3